# 3차시: 대전광역시 공용자전거 (타슈) 데이터 기초 분석

## 2023. 05. 24. 14:10 ~ 16:00 (50분×2)
1. 데이터 분석 흐름
1. 원본 데이터 수집
1. 데이터 전처리
1. 데이터 분석

### 참고자료
- [파이썬 3 표준 문서](https://docs.python.org/3/index.html)
- [대전광역시 시설관리공단](https://www.djsiseol.or.kr/)
- [공공데이터포털](https://www.data.go.kr/)
- [기상청 기상자료개발포털](https://data.kma.go.kr/)

In [ ]:
import datetime

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_rent = pd.read_pickle('tashu_dataset-rental_history_2021.pkl')
df_rent['대여일시'] = pd.to_datetime(df_rent['대여일시'], format='%Y%m%d%H%M%S')
df_rent['반납일시'] = pd.to_datetime(df_rent['반납일시'], format='%Y%m%d%H%M%S')
df_rent

In [ ]:
df_rent.dtypes

In [ ]:
df_station = pd.read_pickle('tashu_dataset-station_information.pkl')
df_station

In [ ]:
df_station.dtypes

## 대여일시 분석

In [ ]:
rent_index = pd.DatetimeIndex(df_rent['대여일시'])

In [ ]:
hourly_df_rent = df_rent['대여일시'].groupby(by=rent_index.hour)
hourly_df_rent.count()

In [ ]:
fig = plt.figure(figsize= (8, 8))
ax = fig.add_subplot()
ax.plot(hourly_df_rent.count())

In [ ]:
print((f'가장 많이 대여하는 시간: '
       f'{hourly_df_rent.count().index[hourly_df_rent.count() == max(hourly_df_rent.count())][0]}'))

In [ ]:
daily_df_rent = df_rent['대여일시'].groupby(by=rent_index.strftime('%Y-%m-%d'))
daily_df_rent.count()

In [ ]:
fig = plt.figure(figsize= (8, 8))
ax = fig.add_subplot()
ax.plot(daily_df_rent.count())